<a href="https://colab.research.google.com/github/thelastgreykoala/time_series/blob/main/TS_DZ1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [8]:
sales_of_company_x = pd.read_csv("monthly-sales-of-company-x-jan-6.csv", index_col = 'Month', parse_dates = True)
monthly_car = pd.read_csv("monthly-car-sales-in-quebec-1960.csv", index_col = 'Month', parse_dates = True)
airlines_passengers = pd.read_csv("international-airline-passengers.csv", index_col = 'Month', parse_dates = True)
